In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import timeit



In [3]:
import torch
from torch.autograd import Variable as V
import torchvision.models as models
from torchvision import transforms as trn
from torch.nn import functional as F
import os
from PIL import Image
from torchsummary import summary

In [4]:
arch = 'resnet50'

model_file = '%s_places365.pth.tar' % arch
if not os.access(model_file, os.W_OK):
    weight_url = 'http://places2.csail.mit.edu/models_places365/' + model_file
    os.system('wget ' + weight_url)

model = models.__dict__[arch](num_classes=365)
checkpoint = torch.load(model_file, map_location=lambda storage, loc: storage)
state_dict = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
model.load_state_dict(state_dict)




<All keys matched successfully>

In [5]:
model.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

     BatchNorm2d-125          [-1, 256, 14, 14]             512
            ReLU-126          [-1, 256, 14, 14]               0
          Conv2d-127         [-1, 1024, 14, 14]         262,144
     BatchNorm2d-128         [-1, 1024, 14, 14]           2,048
            ReLU-129         [-1, 1024, 14, 14]               0
      Bottleneck-130         [-1, 1024, 14, 14]               0
          Conv2d-131          [-1, 256, 14, 14]         262,144
     BatchNorm2d-132          [-1, 256, 14, 14]             512
            ReLU-133          [-1, 256, 14, 14]               0
          Conv2d-134          [-1, 256, 14, 14]         589,824
     BatchNorm2d-135          [-1, 256, 14, 14]             512
            ReLU-136          [-1, 256, 14, 14]               0
          Conv2d-137         [-1, 1024, 14, 14]         262,144
     BatchNorm2d-138         [-1, 1024, 14, 14]           2,048
            ReLU-139         [-1, 1024, 14, 14]               0
      Bottleneck-140         [-1, 1024, 

In [7]:
batch_size = 32
lr = 0.01
momentum = 0.9
weight_decay = 10**(-4)
workers = 0

folder_path =r'C:\Users\duke\Desktop\CS6670_Final_Project\places365standard_easyformat\places365_standard'
traindir = os.path.join(folder_path, 'train')
valdir = os.path.join(folder_path, 'val')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

train_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(traindir, transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize,])),
    batch_size=batch_size, shuffle=True,
    num_workers=workers, pin_memory=True)

val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(valdir, transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize,
    ])),
    batch_size=batch_size, shuffle=False,
    num_workers=workers, pin_memory=True)

# define loss function (criterion) and pptimizer
criterion = nn.CrossEntropyLoss().cuda()

optimizer = torch.optim.SGD(model.parameters(), lr,
                                momentum=momentum,
                                weight_decay=weight_decay)





In [8]:
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [9]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [10]:

print_freq = 1
# batch_time = AverageMeter()
losses = AverageMeter()
top1 = AverageMeter()
top5 = AverageMeter()


model.eval()


for i, (image_input, target) in enumerate(val_loader):
    #image_input = image_input.to('cuda:0')
    image_input = image_input.cuda()
    target = target.cuda()
    image_input_var = torch.autograd.Variable(image_input)
    target_var = torch.autograd.Variable(target)

    # compute output
    output = model(image_input_var)
    loss = criterion(output, target_var)

    # measure accuracy and record loss
    prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
    print(prec1)
    losses.update(loss.data, image_input.size(0))
    top1.update(prec1, image_input.size(0))
    top5.update(prec5, image_input.size(0))

#     # measure elapsed time
#     batch_time.update(time.time() - end)
#     end = time.time()

    if (i % print_freq == 0):
        print('Test: [{0}/{1}]\t'
            'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
            'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
            'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
            i, len(val_loader),  loss=losses,
            top1=top1, top5=top5))

print(' * Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'.format(top1=top1, top5=top5))







tensor(37.5000, device='cuda:0')
Test: [0/1141]	Loss 1.2866 (1.2866)	Prec@1 37.500 (37.500)	Prec@5 96.875 (96.875)
tensor(62.5000, device='cuda:0')
Test: [1/1141]	Loss 1.0724 (1.1795)	Prec@1 62.500 (50.000)	Prec@5 93.750 (95.312)
tensor(59.3750, device='cuda:0')
Test: [2/1141]	Loss 1.3336 (1.2309)	Prec@1 59.375 (53.125)	Prec@5 90.625 (93.750)
tensor(71.8750, device='cuda:0')
Test: [3/1141]	Loss 0.7729 (1.1164)	Prec@1 71.875 (57.812)	Prec@5 96.875 (94.531)
tensor(96.8750, device='cuda:0')
Test: [4/1141]	Loss 0.1638 (0.9259)	Prec@1 96.875 (65.625)	Prec@5 100.000 (95.625)
tensor(84.3750, device='cuda:0')
Test: [5/1141]	Loss 0.6992 (0.8881)	Prec@1 84.375 (68.750)	Prec@5 93.750 (95.312)
tensor(46.8750, device='cuda:0')
Test: [6/1141]	Loss 1.6563 (0.9978)	Prec@1 46.875 (65.625)	Prec@5 87.500 (94.196)
tensor(50., device='cuda:0')
Test: [7/1141]	Loss 1.8938 (1.1098)	Prec@1 50.000 (63.672)	Prec@5 90.625 (93.750)
tensor(50., device='cuda:0')
Test: [8/1141]	Loss 1.4724 (1.1501)	Prec@1 50.000 (62.

Test: [70/1141]	Loss 2.7665 (1.6973)	Prec@1 9.375 (53.433)	Prec@5 71.875 (85.167)
tensor(25., device='cuda:0')
Test: [71/1141]	Loss 2.5556 (1.7092)	Prec@1 25.000 (53.038)	Prec@5 65.625 (84.896)
tensor(18.7500, device='cuda:0')
Test: [72/1141]	Loss 2.7305 (1.7232)	Prec@1 18.750 (52.568)	Prec@5 78.125 (84.803)
tensor(15.6250, device='cuda:0')
Test: [73/1141]	Loss 3.1927 (1.7431)	Prec@1 15.625 (52.069)	Prec@5 53.125 (84.375)
tensor(9.3750, device='cuda:0')
Test: [74/1141]	Loss 2.8503 (1.7578)	Prec@1 9.375 (51.500)	Prec@5 65.625 (84.125)
tensor(53.1250, device='cuda:0')
Test: [75/1141]	Loss 1.5560 (1.7552)	Prec@1 53.125 (51.521)	Prec@5 93.750 (84.252)
tensor(50., device='cuda:0')
Test: [76/1141]	Loss 1.3987 (1.7506)	Prec@1 50.000 (51.502)	Prec@5 96.875 (84.416)
tensor(50., device='cuda:0')
Test: [77/1141]	Loss 1.4543 (1.7468)	Prec@1 50.000 (51.482)	Prec@5 96.875 (84.575)
tensor(50., device='cuda:0')
Test: [78/1141]	Loss 1.5761 (1.7446)	Prec@1 50.000 (51.464)	Prec@5 93.750 (84.691)
tensor(5

Test: [140/1141]	Loss 0.8843 (1.6209)	Prec@1 78.125 (56.184)	Prec@5 93.750 (85.771)
tensor(59.3750, device='cuda:0')
Test: [141/1141]	Loss 1.0905 (1.6171)	Prec@1 59.375 (56.206)	Prec@5 93.750 (85.827)
tensor(71.8750, device='cuda:0')
Test: [142/1141]	Loss 0.8214 (1.6116)	Prec@1 71.875 (56.316)	Prec@5 100.000 (85.927)
tensor(71.8750, device='cuda:0')
Test: [143/1141]	Loss 1.3954 (1.6101)	Prec@1 71.875 (56.424)	Prec@5 84.375 (85.916)
tensor(43.7500, device='cuda:0')
Test: [144/1141]	Loss 1.6870 (1.6106)	Prec@1 43.750 (56.336)	Prec@5 81.250 (85.884)
tensor(28.1250, device='cuda:0')
Test: [145/1141]	Loss 2.3969 (1.6160)	Prec@1 28.125 (56.143)	Prec@5 87.500 (85.895)
tensor(28.1250, device='cuda:0')
Test: [146/1141]	Loss 2.4050 (1.6213)	Prec@1 28.125 (55.952)	Prec@5 87.500 (85.906)
tensor(46.8750, device='cuda:0')
Test: [147/1141]	Loss 2.1391 (1.6248)	Prec@1 46.875 (55.891)	Prec@5 84.375 (85.895)
tensor(50., device='cuda:0')
Test: [148/1141]	Loss 1.6570 (1.6251)	Prec@1 50.000 (55.852)	Prec@5

Test: [210/1141]	Loss 3.9906 (1.6499)	Prec@1 12.500 (54.961)	Prec@5 40.625 (85.441)
tensor(3.1250, device='cuda:0')
Test: [211/1141]	Loss 3.7483 (1.6598)	Prec@1 3.125 (54.717)	Prec@5 56.250 (85.304)
tensor(46.8750, device='cuda:0')
Test: [212/1141]	Loss 2.2445 (1.6626)	Prec@1 46.875 (54.680)	Prec@5 65.625 (85.211)
tensor(78.1250, device='cuda:0')
Test: [213/1141]	Loss 0.6475 (1.6578)	Prec@1 78.125 (54.790)	Prec@5 93.750 (85.251)
tensor(84.3750, device='cuda:0')
Test: [214/1141]	Loss 0.5814 (1.6528)	Prec@1 84.375 (54.927)	Prec@5 96.875 (85.305)
tensor(68.7500, device='cuda:0')
Test: [215/1141]	Loss 1.2684 (1.6511)	Prec@1 68.750 (54.991)	Prec@5 87.500 (85.315)
tensor(46.8750, device='cuda:0')
Test: [216/1141]	Loss 2.0391 (1.6528)	Prec@1 46.875 (54.954)	Prec@5 96.875 (85.369)
tensor(43.7500, device='cuda:0')
Test: [217/1141]	Loss 2.0298 (1.6546)	Prec@1 43.750 (54.903)	Prec@5 84.375 (85.364)
tensor(43.7500, device='cuda:0')
Test: [218/1141]	Loss 2.1180 (1.6567)	Prec@1 43.750 (54.852)	Prec@

Test: [280/1141]	Loss 1.7039 (1.6632)	Prec@1 50.000 (54.738)	Prec@5 96.875 (84.953)
tensor(81.2500, device='cuda:0')
Test: [281/1141]	Loss 1.0252 (1.6609)	Prec@1 81.250 (54.832)	Prec@5 93.750 (84.984)
tensor(71.8750, device='cuda:0')
Test: [282/1141]	Loss 0.9298 (1.6583)	Prec@1 71.875 (54.892)	Prec@5 100.000 (85.038)
tensor(65.6250, device='cuda:0')
Test: [283/1141]	Loss 1.5109 (1.6578)	Prec@1 65.625 (54.930)	Prec@5 87.500 (85.046)
tensor(43.7500, device='cuda:0')
Test: [284/1141]	Loss 2.2023 (1.6597)	Prec@1 43.750 (54.890)	Prec@5 75.000 (85.011)
tensor(34.3750, device='cuda:0')
Test: [285/1141]	Loss 2.3182 (1.6620)	Prec@1 34.375 (54.819)	Prec@5 75.000 (84.976)
tensor(43.7500, device='cuda:0')
Test: [286/1141]	Loss 2.1401 (1.6637)	Prec@1 43.750 (54.780)	Prec@5 71.875 (84.930)
tensor(31.2500, device='cuda:0')
Test: [287/1141]	Loss 2.4517 (1.6664)	Prec@1 31.250 (54.698)	Prec@5 68.750 (84.874)
tensor(34.3750, device='cuda:0')
Test: [288/1141]	Loss 2.2394 (1.6684)	Prec@1 34.375 (54.628)	Pr

...

Test: [841/1141]	Loss 1.6321 (1.8110)	Prec@1 40.625 (52.149)	Prec@5 90.625 (83.406)
tensor(53.1250, device='cuda:0')
Test: [842/1141]	Loss 1.8939 (1.8111)	Prec@1 53.125 (52.150)	Prec@5 78.125 (83.400)
tensor(25., device='cuda:0')
Test: [843/1141]	Loss 2.8829 (1.8124)	Prec@1 25.000 (52.118)	Prec@5 68.750 (83.383)
tensor(25., device='cuda:0')
Test: [844/1141]	Loss 2.9020 (1.8137)	Prec@1 25.000 (52.086)	Prec@5 75.000 (83.373)
tensor(21.8750, device='cuda:0')
Test: [845/1141]	Loss 2.5298 (1.8145)	Prec@1 21.875 (52.050)	Prec@5 68.750 (83.355)
tensor(21.8750, device='cuda:0')
Test: [846/1141]	Loss 3.4224 (1.8164)	Prec@1 21.875 (52.014)	Prec@5 56.250 (83.323)
tensor(31.2500, device='cuda:0')
Test: [847/1141]	Loss 2.5595 (1.8173)	Prec@1 31.250 (51.990)	Prec@5 68.750 (83.306)
tensor(18.7500, device='cuda:0')
Test: [848/1141]	Loss 2.5411 (1.8182)	Prec@1 18.750 (51.951)	Prec@5 78.125 (83.300)
tensor(12.5000, device='cuda:0')
Test: [849/1141]	Loss 3.0644 (1.8196)	Prec@1 12.500 (51.904)	Prec@5 59.3

Test: [911/1141]	Loss 1.0396 (1.8201)	Prec@1 75.000 (51.974)	Prec@5 93.750 (83.220)
tensor(71.8750, device='cuda:0')
Test: [912/1141]	Loss 1.0585 (1.8192)	Prec@1 71.875 (51.995)	Prec@5 93.750 (83.232)
tensor(46.8750, device='cuda:0')
Test: [913/1141]	Loss 2.1508 (1.8196)	Prec@1 46.875 (51.990)	Prec@5 81.250 (83.230)
tensor(43.7500, device='cuda:0')
Test: [914/1141]	Loss 1.8485 (1.8196)	Prec@1 43.750 (51.981)	Prec@5 81.250 (83.227)
tensor(62.5000, device='cuda:0')
Test: [915/1141]	Loss 1.5828 (1.8194)	Prec@1 62.500 (51.992)	Prec@5 81.250 (83.225)
tensor(65.6250, device='cuda:0')
Test: [916/1141]	Loss 0.9497 (1.8184)	Prec@1 65.625 (52.007)	Prec@5 100.000 (83.244)
tensor(53.1250, device='cuda:0')
Test: [917/1141]	Loss 1.2512 (1.8178)	Prec@1 53.125 (52.008)	Prec@5 90.625 (83.252)
tensor(53.1250, device='cuda:0')
Test: [918/1141]	Loss 1.4025 (1.8173)	Prec@1 53.125 (52.010)	Prec@5 90.625 (83.260)
tensor(78.1250, device='cuda:0')
Test: [919/1141]	Loss 0.8345 (1.8163)	Prec@1 78.125 (52.038)	Pr

Test: [981/1141]	Loss 1.3755 (1.7980)	Prec@1 59.375 (52.317)	Prec@5 96.875 (83.564)
tensor(71.8750, device='cuda:0')
Test: [982/1141]	Loss 1.1193 (1.7973)	Prec@1 71.875 (52.337)	Prec@5 96.875 (83.577)
tensor(65.6250, device='cuda:0')
Test: [983/1141]	Loss 1.1172 (1.7967)	Prec@1 65.625 (52.350)	Prec@5 93.750 (83.587)
tensor(56.2500, device='cuda:0')
Test: [984/1141]	Loss 1.7227 (1.7966)	Prec@1 56.250 (52.354)	Prec@5 90.625 (83.595)
tensor(43.7500, device='cuda:0')
Test: [985/1141]	Loss 1.8696 (1.7967)	Prec@1 43.750 (52.345)	Prec@5 93.750 (83.605)
tensor(40.6250, device='cuda:0')
Test: [986/1141]	Loss 2.1907 (1.7971)	Prec@1 40.625 (52.333)	Prec@5 75.000 (83.596)
tensor(81.2500, device='cuda:0')
Test: [987/1141]	Loss 0.7176 (1.7960)	Prec@1 81.250 (52.363)	Prec@5 96.875 (83.610)
tensor(68.7500, device='cuda:0')
Test: [988/1141]	Loss 1.1261 (1.7953)	Prec@1 68.750 (52.379)	Prec@5 90.625 (83.617)
tensor(71.8750, device='cuda:0')
Test: [989/1141]	Loss 1.2548 (1.7947)	Prec@1 71.875 (52.399)	Pre

tensor(68.7500, device='cuda:0')
Test: [1051/1141]	Loss 1.2031 (1.7872)	Prec@1 68.750 (52.626)	Prec@5 96.875 (83.754)
tensor(62.5000, device='cuda:0')
Test: [1052/1141]	Loss 0.9278 (1.7864)	Prec@1 62.500 (52.635)	Prec@5 96.875 (83.767)
tensor(53.1250, device='cuda:0')
Test: [1053/1141]	Loss 1.6924 (1.7863)	Prec@1 53.125 (52.636)	Prec@5 87.500 (83.770)
tensor(68.7500, device='cuda:0')
Test: [1054/1141]	Loss 0.8975 (1.7854)	Prec@1 68.750 (52.651)	Prec@5 93.750 (83.780)
tensor(65.6250, device='cuda:0')
Test: [1055/1141]	Loss 0.8242 (1.7845)	Prec@1 65.625 (52.663)	Prec@5 100.000 (83.795)
tensor(62.5000, device='cuda:0')
Test: [1056/1141]	Loss 1.4544 (1.7842)	Prec@1 62.500 (52.673)	Prec@5 81.250 (83.793)
tensor(46.8750, device='cuda:0')
Test: [1057/1141]	Loss 1.8555 (1.7843)	Prec@1 46.875 (52.667)	Prec@5 81.250 (83.790)
tensor(56.2500, device='cuda:0')
Test: [1058/1141]	Loss 1.9664 (1.7845)	Prec@1 56.250 (52.671)	Prec@5 75.000 (83.782)
tensor(71.8750, device='cuda:0')
Test: [1059/1141]	Loss

Test: [1120/1141]	Loss 1.1629 (1.7678)	Prec@1 62.500 (53.156)	Prec@5 90.625 (83.923)
tensor(65.6250, device='cuda:0')
Test: [1121/1141]	Loss 1.0662 (1.7672)	Prec@1 65.625 (53.167)	Prec@5 93.750 (83.932)
tensor(75., device='cuda:0')
Test: [1122/1141]	Loss 1.2025 (1.7667)	Prec@1 75.000 (53.186)	Prec@5 90.625 (83.938)
tensor(59.3750, device='cuda:0')
Test: [1123/1141]	Loss 1.3655 (1.7663)	Prec@1 59.375 (53.192)	Prec@5 90.625 (83.944)
tensor(68.7500, device='cuda:0')
Test: [1124/1141]	Loss 0.9957 (1.7657)	Prec@1 68.750 (53.206)	Prec@5 93.750 (83.953)
tensor(93.7500, device='cuda:0')
Test: [1125/1141]	Loss 0.2613 (1.7643)	Prec@1 93.750 (53.242)	Prec@5 93.750 (83.961)
tensor(87.5000, device='cuda:0')
Test: [1126/1141]	Loss 0.6553 (1.7633)	Prec@1 87.500 (53.272)	Prec@5 100.000 (83.976)
tensor(90.6250, device='cuda:0')
Test: [1127/1141]	Loss 0.2562 (1.7620)	Prec@1 90.625 (53.305)	Prec@5 100.000 (83.990)
tensor(93.7500, device='cuda:0')
Test: [1128/1141]	Loss 0.3770 (1.7608)	Prec@1 93.750 (53.3

In [11]:
class ResNet50Bottom(nn.Module):
    def __init__(self, original_model):
        super(ResNet50Bottom, self).__init__()
        self.features = nn.Sequential(*list(original_model.children())[:-1])
        
    def forward(self, x):
        x = self.features(x)
        return x


res50_conv = ResNet50Bottom(model)

outputs = res50_conv(image_input_var)

In [28]:
print(image_input_var.size())

torch.Size([20, 3, 224, 224])


In [12]:
print(outputs.size())

torch.Size([20, 2048, 1, 1])
